In [1]:
import os
import pandas as pd

if str(os.getcwdb()[-3:]).split("'")[1] != 'src':
    os.chdir(os.path.dirname(os.getcwdb()))

from utils.cleansing import Cleansing
from utils.modeling import Model


In [2]:
df_diamonds = pd.read_csv(r'data\processed\diamonds_train_1.csv', index_col='id')
df_predict = pd.read_csv(r'data\processed\diamonds_test_1.csv', index_col='id')


# Consideraciones
- Se localiza el tema en Kaggle: https://www.kaggle.com/datasets/shivam2503/diamonds
- Se empieza a trabajar con ese "dataset" (ver los archivos marcados como "UNUSED" y "no competition")
- Se detecta que existe una competición, si bien ya ha terminado: https://www.kaggle.com/competitions/diamonds-part-datamad0122/overview
- Se elige trabajar con los archivos de la competición, cuyas únicas diferencias es que hay un "train" y un "test", y que la variable "target" está escalada
- El "dataset" final es un listado de diamantes con sus características, y el objetivo es predecir el precio
- Se comparará lo obtenido con los resultados de la competición

# EDA
- Los pasos de esta primera parte se detallan de forma más pormenorizada, paso a paso, en el "notebook" titulado "EDA_diamonds"
- En ese "notebook" se hacen dos cosas:
1) Modificaciones esenciales; se liquidan duplicados, se cambia el nombre de las columnas y se pasan las categóricas a numéricas, tanto del "train" como del "test".

2) Modificaciones opcionales; se detectan y ponen a prueba las posibles modifiaciones que llevar a cabo con el "dataframe" de entrenamiento con tal de mejorar el resultado de los modelos. Los resortes de dichos cambios se guardan en forma de funciones (cuando son exclusivos de este proyecto) o clases (cuando es razonable guardarlos para análisis futuros), que se irán llamando a continuación según convenga.

# Modelaje
- Se importan los "dataframes" con las modificaciones esenciales
- Se van intercalando modificiaciones opcionales y diversos modelos hasta dar con el mejor resultado
- Los modelos de prueban en este "notebook" para mayor comodidad, pero se ejecutan sin detallarse en "train.py", desde donde se guardan en la carpeta "model"

## Ronda 1
- Para la primera fase, se prueban todos los modelos sin hacer ninguna modificación adicional
- En esta primera ronda están más detallados los usos de la clase "Model" para que sirva como ejemplo
- Como es de esperar, los resultados no son muy buenos
- Pueden compararse con los de la competición de Kaggle: https://www.kaggle.com/competitions/diamonds-part-datamad0122/leaderboard

In [3]:
# Se crea la instancia de la clase "Model" con la columna "price" como "target"
round_1 = Model(df_diamonds, 'price')


In [4]:
# Se separa el "dataframe" con los parámetros por defecto. Se guardan las porciones por si acaso
X_train, X_test, y_train, y_test = round_1.split_dataframe()


In [5]:
# Se solicitan 10 "folds", del cual se usará el mejor para comparar los modelos y ver cuál llega más lejos
# Como la "target" es de regresión, la instancia seleccionará automáticamente "KFold" en lugar de "StratifiedFold"
round_1_dict = round_1.apply_models(kfolds_num=10)


-- Regression: using best of KFold --
Starting LinearRegression:
- LinearRegression done in 0.43 sec(s). Total time: 0.43
Starting Ridge:
- Ridge done in 0.27 sec(s). Total time: 0.7
Starting DecisionTreeRegressor:
- DecisionTreeRegressor done in 4.69 sec(s). Total time: 5.39
Starting KNeighborsRegressor:
- KNeighborsRegressor done in 11.25 sec(s). Total time: 16.64
Starting RandomForestRegressor:
- RandomForestRegressor done in 264.61 sec(s). Total time: 281.25
Starting SVR:
- SVR done in 388.98 sec(s). Total time: 670.23


In [6]:
# Se guardan los resultados, así como el modelo entrenado, en un diccionario
round_1_dict


{'LinearRegression': {'test': array([8.069, 9.093, 8.297, ..., 9.234, 8.818, 8.368]),
  'prediction': array([8.1751789 , 8.94534424, 7.94609508, ..., 9.13303572, 8.78005708,
         8.22465851]),
  'model': LinearRegression()},
 'Ridge': {'test': array([8.069, 9.093, 8.297, ..., 9.234, 8.818, 8.368]),
  'prediction': array([8.17485364, 8.94499768, 7.94593798, ..., 9.13327901, 8.78003269,
         8.22426296]),
  'model': Ridge()},
 'DecisionTreeRegressor': {'test': array([8.069, 9.093, 8.297, ..., 9.234, 8.818, 8.368]),
  'prediction': array([8.009, 9.24 , 8.107, ..., 9.343, 8.951, 8.328]),
  'model': DecisionTreeRegressor()},
 'KNeighborsRegressor': {'test': array([8.069, 9.093, 8.297, ..., 9.234, 8.818, 8.368]),
  'prediction': array([8.2228, 9.072 , 8.0682, ..., 9.09  , 8.7678, 8.706 ]),
  'model': KNeighborsRegressor()},
 'RandomForestRegressor': {'test': array([8.069, 9.093, 8.297, ..., 9.234, 8.818, 8.368]),
  'prediction': array([8.21693, 9.1298 , 8.13663, ..., 9.27139, 8.90605

In [7]:
# Acto seguido, se miran las métricas
round_1_metrics = round_1.evaluate_metrics(round_1_dict)

round_1_metrics


{'LinearRegression': {'test': array([8.069, 9.093, 8.297, ..., 9.234, 8.818, 8.368]),
  'prediction': array([8.1751789 , 8.94534424, 7.94609508, ..., 9.13303572, 8.78005708,
         8.22465851]),
  'model': LinearRegression(),
  'metrics': {'rmse': 0.222037823248969,
   'mse': 0.0493007949531404,
   'mae': 0.12289980638635319,
   'r2_score': 0.9529090797126748,
   'mape': 0.01587291534328809}},
 'Ridge': {'test': array([8.069, 9.093, 8.297, ..., 9.234, 8.818, 8.368]),
  'prediction': array([8.17485364, 8.94499768, 7.94593798, ..., 9.13327901, 8.78003269,
         8.22426296]),
  'model': Ridge(),
  'metrics': {'rmse': 0.2219231703319744,
   'mse': 0.04924989353019452,
   'mae': 0.1229616970889257,
   'r2_score': 0.9529576995138916,
   'mape': 0.015880870501013333}},
 'DecisionTreeRegressor': {'test': array([8.069, 9.093, 8.297, ..., 9.234, 8.818, 8.368]),
  'prediction': array([8.009, 9.24 , 8.107, ..., 9.343, 8.951, 8.328]),
  'model': DecisionTreeRegressor(),
  'metrics': {'rmse': 0

In [8]:
# Para mejor visualización, se ponen en un "dataframe"
# Las predicciones no son muy buenas, pero "RandomForest" va en cabeza
round_1.create_dataframe(round_1_metrics)


,LinearRegression,Ridge,DecisionTreeRegressor,KNeighborsRegressor,RandomForestRegressor,SVR,BEST,WORST
rmse,0.222038,0.221923,0.130177,0.183865,0.098857,0.209064,RandomForestRegressor,LinearRegression
mse,0.049301,0.049250,0.016946,0.033806,0.009773,0.043708,RandomForestRegressor,LinearRegression
mae,0.122900,0.122962,0.088797,0.135156,0.067398,0.127477,KNeighborsRegressor,KNeighborsRegressor
r2_score,0.952909,0.952958,0.983814,0.967709,0.990665,0.958252,RandomForestRegressor,LinearRegression
mape,0.015873,0.015881,0.011432,0.017806,0.008719,0.016523,KNeighborsRegressor,KNeighborsRegressor


## Ronda 2
- Se repite la ronda 1, pero esta vez se escalan las variables